In [ ]:
#git clone https://github.com/lucidrains/iTransformer.git

In [1]:
#%pip install iTransformer
#!python3 -m pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117
#!pip install holidays

from iTransformer import iTransformer
import torch
import pandas as pd
from pathlib import Path
import holidays
import glob

import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from beartype.typing import Union, Tuple
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from utils.helpers import SlidingWindowTimeSeriesDataset, ModelConfig, create_checkpoint, calc_percentiles


/home/ben_ten/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ben_ten/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# iTransformer from paper, difficult to use // dataprep

In [15]:
#use of paper iTransformer

from model.iTransformer import Model 

from types import SimpleNamespace

configs = SimpleNamespace(
    seq_len=100,
    pred_len=10,
    output_attention=True,
    d_model=512,
    embed='some_embedding_config',
    freq='H',  # e.g., hourly frequency
    dropout=0.1,
    class_strategy='some_strategy',
    factor=5,
    e_layers=6,
    n_heads=8,
    d_ff=2048,
    activation='relu'  # e.g., relu activation
)

model = Model(configs)


from dataclasses import dataclass

@dataclass
#class ModelConfig:
	seq_len: int = 96
	pred_len: int = 24
	output_attention: int =  
	d_model
	embed
	freq
	dropout
	class_strategy
	factor
	n_heads
	d_ff
	activation
	e_layers



In [3]:
#passenger_df = pd.read_csv(Path("/home/ben_ten/ben/MA/air_passenger/AirPassengers.csv"), delimiter=",")

electricity = pd.read_csv(Path("/home/ben_ten/ben/MA/LD2011_2014.csv"), delimiter=",")
electricity.drop(['categorical_hour', 'categorical_day_of_week', 'hours_from_start', 'Unnamed: 0', 'categorical_id', 'date'], axis=1, inplace=True)
for i in electricity["id"].unique():
	if electricity[electricity["id"]==i]["days_from_start"].min() != 1096:
		electricity.drop(electricity[electricity["id"] == i].index, inplace=True)
electricity.to_csv("electricity_small.csv", index=False)
electricity

,power_usage,time_idx,days_from_start,id,hour,day,day_of_week,month
0,2.538071,26304.0,1096,MT_001,0,1,2,1
1,2.855330,26305.0,1096,MT_001,1,1,2,1
2,2.855330,26306.0,1096,MT_001,2,1,2,1
3,2.855330,26307.0,1096,MT_001,3,1,2,1
4,2.538071,26308.0,1096,MT_001,4,1,2,1
...,...,...,...,...,...,...,...,...
2198067,20824.324324,32299.0,1345,MT_370,19,7,6,9
2198068,19527.027027,32300.0,1345,MT_370,20,7,6,9
2198069,20202.702703,32301.0,1345,MT_370,21,7,6,9
2198070,19851.351351,32302.0,1345,MT_370,22,7,6,9


In [ ]:
#dataprep pessenger

# Sample pandas series
passenger = passenger_df['#Passengers']

# Convert the pandas series to a NumPy array
numpy_arr = passenger.to_numpy()

# Reshape the NumPy array to have shape (1, Series, 1)
reshaped_array = numpy_arr.reshape(1, -1, 1)

# Convert the reshaped NumPy array to a PyTorch tensor
torch_tensor = torch.Tensor(reshaped_array)

# Check the shape of the PyTorch tensor
print(torch_tensor.shape)

def min_max_scaling(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    normalized_tensor = (tensor - min_val) / (max_val - min_val)
    return normalized_tensor

def standard_scaling(tensor):
    mean = tensor.mean(dim=(0, 1), keepdim=True)
    std = tensor.std(dim=(0, 1), unbiased=False, keepdim=True)  # using unbiased=False for sample standard deviation
    scaled_tensor = (tensor - mean) / (std + 1e-7) 
    return scaled_tensor

#uncommend for normalized/standardized tensors
#torch_tensor = min_max_scaling(torch_tensor)
torch_tensor = standard_scaling(torch_tensor)

tensor_x = torch_tensor[:, :132, :]
tensor_y = torch_tensor[:, 132:, :]

# Code for electricity and training

In [2]:
# execute this to fully prep 1 batch electricity for iTrasnfromer 

def split_data(df, valid_boundary=1315, test_boundary=1339, use_scaler=True):
	index = df['days_from_start']
	train = df.loc[index < valid_boundary]
	valid = df.loc[(index >= valid_boundary - 7) & (index < test_boundary)]
	test = df.loc[index >= test_boundary - 7]

	if use_scaler == True:
		electricity_scaler = StandardScaler()
	
		variates =  ['power_usage', 'time_idx', 'days_from_start', 'hour', 'day', 'day_of_week', 'month']
		electricity_scaler.fit(train[variates])
		train[variates] = electricity_scaler.transform(train[variates])
		test[variates] = electricity_scaler.transform(test[variates])
		valid[variates] = electricity_scaler.transform(valid[variates])

		electricity_dict = {
						"train" : train,
						"test" : test,
						"valid" : valid,
						"scaler" : electricity_scaler
						}
	else:
		electricity_dict = {
						"train" : train,
						"test" : test,
						"valid" : valid,
						}
	return electricity_dict

def df_to_tensor(df):
	df.sort_values(by=['id', 'time_idx'], inplace=True)

	# Initialize a list to store tensors
	tensors = []

	# Unique IDs
	unique_ids = df['id'].unique()

	# For each 'id', create a tensor and add it to the list
	for unique_id in unique_ids:
		# Filter rows by 'id'
		df_id = df[df['id'] == unique_id]    
		# Select relevant columns and convert to tensor
		tensor = torch.tensor(df_id[['power_usage', 'hour', 'day', 'day_of_week', 'month']].values, dtype=torch.float32)
		# Add tensor to list
		tensors.append(tensor)

	# Concatenate all tensors along a new dimension
	result_tensor = torch.stack(tensors)
	return result_tensor



df = pd.read_csv('datasets/electricity/electricity_small.csv')
electricity = split_data(df, use_scaler=True)

train_tensor = df_to_tensor(electricity["train"])
valid_tensor = df_to_tensor(electricity["valid"])
test_tensor = df_to_tensor(electricity["test"])

/tmp/ipykernel_20301/201167750.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[variates] = electricity_scaler.transform(train[variates])
/tmp/ipykernel_20301/201167750.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[variates] = electricity_scaler.transform(test[variates])
/tmp/ipykernel_20301/201167750.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [3]:
# small multi batch dataset for trial
train_tensor_small = train_tensor[:1, 0:, :] # 10 ids and 1000 timesteps -> 10 * (1000/144) = 60 timeseries
valid_tensor_small = valid_tensor[:20, 0:, :]
test_tensor_small = test_tensor[:, 0:, :]


valid_tensor_id_1 = valid_tensor[:1, 0:, :]
valid_tensor_id_2 =valid_tensor[2:3, 0:, :]

# define input and output size for dataset creation and model config
window_size = 132
pred_length = 12

# create dataset with mutiple input timeseries per id accodring to window_length/pred_length
train_sliding_window = SlidingWindowTimeSeriesDataset(train_tensor_small, window_size, pred_length)
valid_sliding_window = SlidingWindowTimeSeriesDataset(valid_tensor_small, window_size, pred_length)
test_sliding_window = SlidingWindowTimeSeriesDataset(test_tensor_small, window_size, pred_length)

train_dataloader = DataLoader(train_sliding_window, batch_size=64, shuffle=True, num_workers=5)
valid_dataloader = DataLoader(valid_sliding_window, batch_size=32, shuffle=False)
test_datalaoder = DataLoader(test_sliding_window, batch_size=32, shuffle=False)

print(f"Datalaoder n_batches:\nTrain: {len(train_dataloader)}\nValid: {len(valid_dataloader)}\nTest: {len(test_datalaoder)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# defining all needed instances
config = ModelConfig(lookback_len=window_size, pred_length=pred_length)
model = iTransformer(config).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
writer = SummaryWriter(log_dir='logs')


Datalaoder n_batches:
Train: 80
Valid: 376
Test: 2111
Using device: cpu


In [4]:
epoch = 3

def train_one_epoch(epoch, model, device, train_dataloader, optimizer, scheduler, writer):
	global_step = 0
	model.train()
	total_loss = 0
	for input, target_covariates in tqdm(train_dataloader, desc=f"Epoch: {epoch}"):
		optimizer.zero_grad()
		input = input.to(device)
		output_covariates = model(input)

		# this can be used for target specific fine-tuning
		output = output_covariates[12][:,:,0]
		target = target_covariates[:,:,0]

		loss = torch.nn.MSELoss()
		loss = loss(output_covariates[12], target_covariates)  # compute loss on all variates
		#computed_loss = loss(output, target)  # compute loss on target variate

		loss.backward()
		optimizer.step()
		total_loss += loss.item()

		writer.add_scalar('Loss/train', loss, global_step)
		writer.add_scalar('LR/train', optimizer.param_groups[0]['lr'], global_step)

		global_step+=1

	print(f'Epoch {epoch}, Loss: {total_loss / len(train_dataloader)}')

	scheduler.step()

	writer.close()
	create_checkpoint(model, optimizer, scheduler, epoch, loss, global_step, "first_20_ids_full")

for epoch in range(1, epoch + 1):
	train_one_epoch(epoch, model, device, train_dataloader, optimizer, scheduler, writer)


Epoch: 1: 100%|██████████| 80/80 [00:19<00:00,  4.13it/s]


Epoch 1, Loss: 0.17024583357851952
Checkpointing succesfull after epoch 1


Epoch: 2: 100%|██████████| 80/80 [00:19<00:00,  4.02it/s]


Epoch 2, Loss: 0.03198554607806727
Checkpointing succesfull after epoch 2


Epoch: 3: 100%|██████████| 80/80 [00:19<00:00,  4.11it/s]

Epoch 3, Loss: 0.0240678658883553
Checkpointing succesfull after epoch 3


In [38]:
import numpy as np

def calc_percentiles(predictions, actuals, percentile):
	# take predictions, actuals in correct order and return percentiles
	# return p10, p50, p90
	
	errors = np.abs(predictions - actuals)

	# Sort the errors
	sorted_errors = np.sort(errors)

# Calculate p10, P50 and P90
	p_percentile = np.percentile(sorted_errors, percentile)
	return p_percentile

In [4]:
valid_tensor_id_1 = valid_tensor[:1, 0:, :]
valid_tensor_id_2 =valid_tensor[2:3, 0:, :]



window_size = 132
pred_length = 12

# create dataset with mutiple input timeseries per id accodring to window_length/pred_length
valid_sliding_window_id1 = SlidingWindowTimeSeriesDataset(valid_tensor_id_1, window_size, pred_length)
valid_sliding_window_id2 = SlidingWindowTimeSeriesDataset(valid_tensor_id_2, window_size, pred_length)

valid_dataloader_id1 = DataLoader(valid_sliding_window_id1, batch_size=32, shuffle=False)
valid_dataloader_id2 = DataLoader(valid_sliding_window_id2, batch_size=32, shuffle=False)

In [6]:
eval_function(valid_dataloader_id1)

Epoch: Validating:  21%|██        | 4/19 [00:00<00:00, 17.89it/s]

Epoch: Validating: 100%|██████████| 19/19 [00:00<00:00, 21.54it/s]

MSE Loss: 0.04394805421562571
P10: 0.0032778435558276746
	P50: 0.026434942344693763
P90: 0.4216238235172473


In [8]:
eval_function(valid_dataloader_id2)

Epoch: Validating:  11%|█         | 2/19 [00:00<00:01, 13.90it/s]

Epoch: Validating: 100%|██████████| 19/19 [00:01<00:00, 15.65it/s]

MSE Loss: 0.043910691220509376
P10: 0.0032440845952614364
	P50: 0.02638869992408313
P90: 0.42143149250432066


In [5]:
def eval_function(valid_dataloader):
	# loading saved model
#	model = iTransformer(config)

	checkpoint = torch.load("model_name_first_20_ids_full_epoch_3.pt")
	model.load_state_dict(checkpoint['model_state_dict'])
	optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
	start_epoch = checkpoint['epoch'] + 1  # Start training from 
	scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
	scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
	writer = SummaryWriter(log_dir='logs')
	global_step = checkpoint['global_step_writer']

	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	model.to(device)
	model.eval()
	global_step_eval = 1
	total_p10_eval = 0
	total_p50_eval = 0
	total_p90_eval = 0

	# Example inference code
	with torch.no_grad():
		total_loss_eval = 0
		for input, target_covariates in tqdm(valid_dataloader, desc=f"Epoch: Validating"):
			input = input.to(device)
			output_covariates = model(input)

			# this can be used for target specific fine-tuning
			#output = output_covariates[12][:,:,0]
			#target = target_covariates[:,:,0]

			loss = torch.nn.MSELoss()
			loss = loss(output_covariates[12], target_covariates)  # compute loss on all variates
			#computed_loss = loss(output, target)  # compute loss on target variate

			total_loss_eval += loss.item()
			total_p10_eval += calc_percentiles(output_covariates[12], target_covariates, 10)
			total_p50_eval += calc_percentiles(output_covariates[12], target_covariates, 50)
			total_p90_eval += calc_percentiles(output_covariates[12], target_covariates, 90)


	print(f'MSE Loss: {total_loss_eval / len(valid_dataloader)}\nP10: { total_p10_eval/ len(valid_dataloader)}\n\
	P50: {total_p50_eval/ len(valid_dataloader)}\nP90: { total_p90_eval/ len(valid_dataloader)}')

	writer.add_scalar('MSE/valid', total_loss_eval/ len(valid_dataloader), global_step_eval)
	writer.add_scalar('P10/valid', total_p10_eval/ len(valid_dataloader), global_step_eval)
	writer.add_scalar('P50/valid', total_p50_eval/ len(valid_dataloader), global_step_eval)
	writer.add_scalar('P90/valid', total_p90_eval/ len(valid_dataloader), global_step_eval)

	writer.close()